In [1]:
import pandas as pd
import glob
import os
import re
import ast
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import gc
from sklearn.preprocessing import MinMaxScaler

# Xóa những dòng thiếu giá trị

##### Xử lý nhiều file

In [2]:
# === PHẦN 1: Làm sạch dữ liệu File_loc ===
# Định nghĩa thư mục đầu vào và mẫu file
input_dir = 'Doc_file_Jsonl/File_loc'
pattern = os.path.join(input_dir, 'File_loc_batch_*.csv')

# Lặp qua các file CSV để làm sạch
for file in glob.glob(pattern):
    print(f"Đang xử lý: {file}")
    df = pd.read_csv(file, dtype={'name': str, 'abstract': str, 'sections': str})
    original_len = len(df)
    # Xóa các dòng thiếu giá trị ở cột name, abstract, sections
    df = df.dropna(subset=['name', 'abstract', 'sections'])
    # Kiểm tra tính hợp lệ của sections (phải là danh sách)
    valid_sections = df['sections'].apply(lambda x: isinstance(ast.literal_eval(x), list) if pd.notnull(x) else False)
    df = df[valid_sections]
    # Ghi đè file gốc
    df.to_csv(file, index=False)
    print(f"Đã cập nhật: {file} (xóa {original_len - len(df)} dòng)")

Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_1.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_1.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_10.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_10.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_11.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_11.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_12.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_12.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_13.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_13.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_14.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_14.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_15.csv
Đã cập nhật: Doc_file_Jsonl/File_loc\File_loc_batch_15.csv (xóa 0 dòng)
Đang xử lý: Doc_file_Jsonl/File_loc\File_loc_batch_16.csv
Đã cập nhật: Doc_file_Jsonl/File_l

##### Xử lý 1 file

In [3]:
# # === PHẦN 1: Làm sạch dữ liệu File_loc ===
# # Định nghĩa thư mục đầu vào và mẫu file

# input_dir = 'Doc_file_Jsonl/File_loc'
# file = os.path.join(input_dir, 'File_loc_batch_1.csv')

# print(f"Đang xử lý: {file}")
# df = pd.read_csv(file, dtype={'name': str, 'abstract': str, 'sections': str})
# original_len = len(df)
# # Xóa các dòng thiếu giá trị ở cột name, abstract, sections
# df = df.dropna(subset=['name', 'abstract', 'sections'])
# # Kiểm tra tính hợp lệ của sections (phải là danh sách)
# valid_sections = df['sections'].apply(lambda x: isinstance(ast.literal_eval(x), list) if pd.notnull(x) else False)
# df = df[valid_sections]
# # Ghi đè file gốc
# df.to_csv(file, index=False)
# print(f"Đã cập nhật: {file} (xóa {original_len - len(df)} dòng)")

# Tính điểm tổng hợp độ quan trọng

##### Tính điểm nhiều file

In [4]:
# === PHẦN 2: Tính điểm TF-IDF ===
# Hàm trích xuất văn bản từ sections
def extract_text_from_sections(sections):
    text_parts = []
    if isinstance(sections, list):
        for section in sections:
            if 'name' in section:
                text_parts.append(str(section['name']))
            if 'has_parts' in section:
                for part in section['has_parts']:
                    if isinstance(part, dict) and 'value' in part:
                        text_parts.append(str(part['value']))
    return ' '.join(text_parts)

# Hàm kết hợp văn bản từ name, abstract và sections
def combine_text(row):
    name = row['name'] if pd.notna(row['name']) else ""
    abstract = row['abstract'] if pd.notna(row['abstract']) else ""
    try:
        sections = ast.literal_eval(row['sections']) if pd.notna(row['sections']) else []
        sections_text = extract_text_from_sections(sections)
    except Exception:
        sections_text = ""
    return f"{name} {abstract} {sections_text}"

# Tạo thư mục lưu điểm TF-IDF
output_dir = 'Diem_TF-IDF'
os.makedirs(output_dir, exist_ok=True)

# Từ điển lưu điểm TF-IDF của các title
valid_names_scores = {}

# Xử lý từng file CSV để tính TF-IDF
for filename in sorted(os.listdir(input_dir)):
    if filename.startswith('File_loc_batch_') and filename.endswith('.csv'):
        filepath = os.path.join(input_dir, filename)
        print(f"🔍 Xử lý {filename}...")
        df = pd.read_csv(filepath, dtype={'name': str, 'abstract': str, 'sections': str})
        # Tạo cột full_text từ name, abstract, sections
        df['full_text'] = df.apply(combine_text, axis=1)
        # Tính TF-IDF
        vectorizer = TfidfVectorizer(max_features=1000)
        tfidf_matrix = vectorizer.fit_transform(df['full_text'])
        df['tfidf_score'] = tfidf_matrix.mean(axis=1).A1
        # Chuẩn hóa điểm TF-IDF về [0,1]
        scaler = MinMaxScaler()
        df['tfidf_score'] = scaler.fit_transform(df[['tfidf_score']])
        # Lấy số batch từ tên file
        batch_num = re.search(r'batch_(\d+)', filename).group(1)
        output_path = os.path.join(output_dir, f'Diem_TF-IDF_batch_{batch_num}.csv')
        # Lưu kết quả TF-IDF
        df[['name', 'tfidf_score']].to_csv(output_path, index=False)
        # Lưu điểm TF-IDF vào từ điển
        for _, row in df.iterrows():
            valid_names_scores[row['name']] = row['tfidf_score']
        print(f"✅ Đã lưu: {output_path}")

🔍 Xử lý File_loc_batch_1.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_1.csv
🔍 Xử lý File_loc_batch_10.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_10.csv
🔍 Xử lý File_loc_batch_11.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_11.csv
🔍 Xử lý File_loc_batch_12.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_12.csv
🔍 Xử lý File_loc_batch_13.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_13.csv
🔍 Xử lý File_loc_batch_14.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_14.csv
🔍 Xử lý File_loc_batch_15.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_15.csv
🔍 Xử lý File_loc_batch_16.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_16.csv
🔍 Xử lý File_loc_batch_17.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_17.csv
🔍 Xử lý File_loc_batch_18.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_18.csv
🔍 Xử lý File_loc_batch_19.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_19.csv
🔍 Xử lý File_loc_batch_2.csv...
✅ Đã lưu: Diem_TF-IDF\Diem_TF-IDF_batch_2.csv
🔍 Xử lý File_loc_batch_20.csv...
✅ Đã lưu: D

##### Tính điểm 1 file

In [5]:
# # === PHẦN 2: Tính điểm TF-IDF ===
# # Hàm trích xuất văn bản từ sections
# def extract_text_from_sections(sections):
#     text_parts = []
#     if isinstance(sections, list):
#         for section in sections:
#             if 'name' in section:
#                 text_parts.append(str(section['name']))
#             if 'has_parts' in section:
#                 for part in section['has_parts']:
#                     if isinstance(part, dict) and 'value' in part:
#                         text_parts.append(str(part['value']))
#     return ' '.join(text_parts)

# # Hàm kết hợp văn bản từ name, abstract và sections
# def combine_text(row):
#     name = row['name'] if pd.notna(row['name']) else ""
#     abstract = row['abstract'] if pd.notna(row['abstract']) else ""
#     try:
#         sections = ast.literal_eval(row['sections']) if pd.notna(row['sections']) else []
#         sections_text = extract_text_from_sections(sections)
#     except Exception:
#         sections_text = ""
#     return f"{name} {abstract} {sections_text}"

# # Tạo thư mục lưu điểm TF-IDF
# output_dir = 'Diem_TF-IDF'
# os.makedirs(output_dir, exist_ok=True)

# # Từ điển lưu điểm TF-IDF của các title
# valid_names_scores = {}

# # Xử lý file CSV để tính TF-IDF
# filename = os.path.basename(file)
# print(f"🔍 Xử lý {filename}...")
# df = pd.read_csv(file, dtype={'name': str, 'abstract': str, 'sections': str})
# # Tạo cột full_text từ name, abstract, sections
# df['full_text'] = df.apply(combine_text, axis=1)
# # Tính TF-IDF
# vectorizer = TfidfVectorizer(max_features=1000)
# tfidf_matrix = vectorizer.fit_transform(df['full_text'])
# df['tfidf_score'] = tfidf_matrix.mean(axis=1).A1
# # Chuẩn hóa điểm TF-IDF về [0,1]
# scaler = MinMaxScaler()
# df['tfidf_score'] = scaler.fit_transform(df[['tfidf_score']])
# # Lấy số batch từ tên file
# batch_num = re.search(r'batch_(\d+)', filename).group(1)
# output_path = os.path.join(output_dir, f'Diem_TF-IDF_batch_{batch_num}.csv')
# # Lưu kết quả TF-IDF
# df[['name', 'tfidf_score']].to_csv(output_path, index=False)
# # Lưu điểm TF-IDF vào từ điển
# for _, row in df.iterrows():
#     valid_names_scores[row['name']] = row['tfidf_score']
# print(f"✅ Đã lưu: {output_path}")

## File Crawl

##### Xử lý 1 batch

In [9]:
# === PHẦN 3: Xử lý file Crawl ===
# Định nghĩa thư mục và cấu hình
raw_folder = "Crawl(2)/Crawl_raw"
output_folder = os.path.join("Crawl(2)", "Crawl_ca_nam_long")

# Tạo thư mục nếu chưa có
os.makedirs(output_folder, exist_ok=True)

# Xử lý một file cụ thể
file = os.path.join(raw_folder, "Crawl_raw_batch_4.csv")
batch_number = 4

if not os.path.exists(file):
    raise FileNotFoundError(f"Không tìm thấy file: {file}")

print(f"\n=== Xử lý file: {os.path.basename(file)} ===")

# Đọc và lọc dữ liệu
df = pd.read_csv(file, dtype={'title': str, 'views': 'float32'})
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')
df.dropna(subset=['date'], inplace=True)

original_len = len(df)
df = df[df['title'].isin(valid_names_scores.keys())]
print(f"Batch {batch_number}: Lọc từ {original_len} → {len(df)} dòng")

if df.empty:
    print(f"Batch {batch_number}: Bỏ qua vì không có title nào khớp")
else:
    # === Lọc các title có số ngày bị thiếu dưới 30% ===
    # Tính khoảng ngày mong đợi (từ ngày nhỏ nhất đến lớn nhất trong dữ liệu)
    min_date = df['date'].min()
    max_date = df['date'].max()
    expected_days = (max_date - min_date).days + 1  # Tổng số ngày mong đợi

    # Tính số ngày thực tế và tỷ lệ thiếu cho mỗi title
    title_day_counts = df.groupby('title')['date'].nunique().reset_index(name='actual_days')
    title_day_counts['missing_days'] = expected_days - title_day_counts['actual_days']
    title_day_counts['missing_ratio'] = title_day_counts['missing_days'] / expected_days

    # Lọc các title có tỷ lệ thiếu dưới 30%
    valid_titles = title_day_counts[title_day_counts['missing_ratio'] < 0.3]['title']
    df = df[df['title'].isin(valid_titles)]
    print(f"Batch {batch_number}: Sau khi lọc missing days < 30%, còn {len(df)} dòng ({len(valid_titles)} titles)")

    if df.empty:
        print(f"Batch {batch_number}: Bỏ qua vì không có title nào thỏa mãn điều kiện missing days")
    else:
        # Thêm đặc trưng cho mô hình
        df['day_of_week'] = df['date'].dt.dayofweek.astype(np.int8)
        df['month'] = df['date'].dt.month.astype(np.int8)
        df['quarter'] = df['date'].dt.quarter.astype(np.int8)
        df['tfidf_score'] = df['title'].map(valid_names_scores)

        # Kiểm tra dữ liệu trước khi lưu
        print(f"🔍 Cột trước khi lưu: {list(df.columns)}")
        print(f"🔍 Dữ liệu mẫu:\n{df.head()}")

        # Lưu dữ liệu long cho cả ba mô hình (ARIMA, TFT, Informer)
        output_path = os.path.join(output_folder, f"Crawl_full_views_ca_nam_batch_{batch_number}.csv")
        df[['date', 'title', 'views', 'day_of_week', 'month', 'quarter', 'tfidf_score']].to_csv(output_path, index=False)
        print(f"✅ Đã lưu file long cho ARIMA, TFT, Informer: {output_path} ({len(df)} dòng)")

print("🎉 Hoàn tất xử lý file!")


=== Xử lý file: Crawl_raw_batch_4.csv ===
Batch 4: Lọc từ 47640550 → 47538893 dòng
Batch 4: Sau khi lọc missing days < 30%, còn 38751822 dòng (10605 titles)
🔍 Cột trước khi lưu: ['title', 'date', 'views', 'day_of_week', 'month', 'quarter', 'tfidf_score']
🔍 Dữ liệu mẫu:
                  title       date  views  day_of_week  month  quarter  \
672  Rossa Mediterranea 2024-01-01    2.0            0      1        1   
673  Rossa Mediterranea 2024-01-02    1.0            1      1        1   
674  Rossa Mediterranea 2024-01-03    2.0            2      1        1   
675  Rossa Mediterranea 2024-01-04    1.0            3      1        1   
676  Rossa Mediterranea 2024-01-05    0.0            4      1        1   

     tfidf_score  
672     0.465878  
673     0.465878  
674     0.465878  
675     0.465878  
676     0.465878  
✅ Đã lưu file long cho ARIMA, TFT, Informer: Crawl(2)\Crawl_ca_nam_long\Crawl_full_views_ca_nam_batch_4.csv (38751822 dòng)
🎉 Hoàn tất xử lý file!
